In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,SincConv_fast,LambdaLayer,biasLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [162]:
## change ##
batch_size = 32
clip = .1
block = ResidualBlock
normalFun = normalize2

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalFun(train[:,0])
# without tanh is worse-off

Prepare data

In [7]:
# train_gen = SequenceGen(train)
# train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

# val_gen = SequenceGen(train,False)
# val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

In [131]:
train_gen = SequenceGenSample(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGenSample(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [146]:
model = Sequential(SincConv_fast(64,128,sample_rate=100000,stride=64),
                      block(64),
                      Dropout(0.5),
                      block(128),
                      Dropout(0.5),                 
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      Linear(256,1,bias=False),
                      LeakyReLU(0.2,True),
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                      #LambdaLayer(lambda x:x.squeeze(2).median(1,keepdim=True)[0]),                   
                      biasLayer((1,)),
                      #Linear(1,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [147]:
opt = Adam(trainable_parameter(model),lr=4e-4)

In [148]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [149]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:4.5943815524761495, val_loss:4.678011894226074
epoch:1, train_loss:3.096679225564003, val_loss:3.295793294906616
epoch:2, train_loss:2.2941563840095816, val_loss:3.318624258041382
epoch:3, train_loss:2.1509709667700987, val_loss:2.5512492656707764
epoch:4, train_loss:2.092960469997846, val_loss:2.4404022693634033
Training completed in 9.100494623184204s


In [151]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip,lossBest=2.44)

epoch:0, train_loss:2.0531784983781667, val_loss:2.588535785675049
epoch:1, train_loss:2.0186399668455124, val_loss:2.4619321823120117
epoch:2, train_loss:2.0004457178024144, val_loss:2.3914332389831543
epoch:3, train_loss:1.9703149016086872, val_loss:2.677093744277954
epoch:4, train_loss:1.943378393466656, val_loss:2.5897536277770996
Training completed in 8.98228120803833s


Submission

In [163]:
submission = pd.read_csv('../Data/sample_submission.csv',)
test_gen = SequenceGenTestSample(submission.seg_id.tolist(),normalFun)
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)
yhat = predict(model,test_gen)
submission.iloc[:,1] = np.median(yhat.reshape(-1,4),1)
submission.to_csv('../Submission/sub_raw_shareWeight_sample_SincNet.csv',index=False)